In [1]:
from numpy import sort
import vectorbt as vbt
import pandas as pd 
import datetime, pytz
from datetime import datetime, timedelta

# Read In Data

In [2]:
# Enter your parameters here

start_date = datetime(2018, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
# end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)
# The following is the number of days to look back for the analysis
# time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window


vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
# vbt.settings.portfolio['fees'] = 0.0025  # 0.25% # These may already be incorporated into trade file 
# vbt.settings.portfolio['commission'] = 0.0025  # 0.25% May want to use this later

# Read in the Trade File from Tradingview Strategy

In [6]:
# Download trade file from tradingview strategy as csv
trade_file = "./data/Dynamic_Wavebased_1.0_Strategy_2022-02-06 (2).csv" 
df_trades = pd.read_csv(trade_file)

print(df_trades)

      Trade #         Type Signal         Date/Time    Price  Contracts  \
0           1  Entry Short  ParSE  2018-01-01 16:00   748.27    80.1282   
1           1   Exit Short  ParLE  2018-01-01 20:00   761.69    80.1282   
2           2   Entry Long  ParLE  2018-01-01 20:00   761.69    78.8924   
3           2    Exit Long    tpL  2018-01-02 00:00   799.65    78.8924   
4           3   Entry Long  ParLE  2018-01-26 18:00  1061.31    57.5914   
...       ...          ...    ...               ...      ...        ...   
1255      628    Exit Long    tpL  2022-01-31 16:00  2686.34   645.9219   
1256      629  Entry Short  ParSE  2022-02-02 16:00  2663.90   328.7534   
1257      629   Exit Short  ParLE  2022-02-03 20:00  2665.52   328.7534   
1258      630   Entry Long  ParLE  2022-02-03 20:00  2665.52   333.3828   
1259      630    Exit Long    tpL  2022-02-04 06:00  2798.52   333.3828   

      Profit NONE  Profit %  Cum. Profit NONE  Cum. Profit %  Run-up NONE  \
0        -1099.52     

Working on Finding out about leverage


In [ ]:

# print(df_trades)
print(df_trades.columns)

subset = df_trades[["Type", "Price", "Contracts", "Cum. Profit NONE", "Profit %"]]
# print(df_trades)
only_entry = subset[subset["Type"].str.contains("Entry")]

only_entry["total_value"] = only_entry["Cum. Profit NONE"] + 100000
only_entry["payment"] = only_entry["Price"] * only_entry["Contracts"]
only_entry["leverage"] = only_entry["payment"]/only_entry["total_value"]
only_entry["total_value"] = only_entry["total_value"].shift(1)

# == "Entry Long" | subset["Type"] == "Entry Short"]
# only_entry
# pd.set_option("display.max_rows", None, "display.max_columns", None)
only_entry.to_csv("leverage_finding.csv")
print(only_entry)

In [7]:
# get binance data doing it this way allows for you to update your data rather than re-downloading it
symbol = "ETHUSDT"
freq = '2h' # My strategy was based on 2hour candles for ETHUSDT
binance_data = vbt.BinanceData.download(symbol, start=start_date, end="now UTC", interval=freq, show_progress=True) 
try: 
    binance_data.load("./data/BINANCE_ETHUSDT_2h.pickle")
    print("Loaded data from file")
except: 
    print("loading data from pickle failed")
    binance_data.get() # get data from binance first time can comment this out if you want to use the data from the csv
    binance_data.save("./data/BINANCE_ETHUSDT_2h.pickle") # save data to file this is a pickle file 
# binance_data.update() # update the data
df_data = binance_data.data["ETHUSDT"] # get the dataframe from the binance data object



2018-01-01 00:00:00+00:00 - 2022-02-11 22:00:00+00:00: : 37it [00:29,  1.25it/s]

Loaded data from file


In [ ]:
# If you would like to update the data, you can do it here
binance_data.update(show_progress=True)
print("Updated data")
binance_data.save("./data/BINANCE_ETHUSDT_2h.pickle")
print("Saved data")

# Fix up the tradingview trade dataframe

In [8]:
df_data.index = df_data.index.tz_convert(None) # convert the index to tz-naive
# The following only needs to be done once after reading the trades in from tradingview
df_trades = df_trades.set_index('Date/Time')
df_trades.index = pd.to_datetime(df_trades.index)

# print(df_data)
print(df_trades)


                     Trade #         Type Signal    Price  Contracts  \
Date/Time                                                              
2018-01-01 16:00:00        1  Entry Short  ParSE   748.27    80.1282   
2018-01-01 20:00:00        1   Exit Short  ParLE   761.69    80.1282   
2018-01-01 20:00:00        2   Entry Long  ParLE   761.69    78.8924   
2018-01-02 00:00:00        2    Exit Long    tpL   799.65    78.8924   
2018-01-26 18:00:00        3   Entry Long  ParLE  1061.31    57.5914   
...                      ...          ...    ...      ...        ...   
2022-01-31 16:00:00      628    Exit Long    tpL  2686.34   645.9219   
2022-02-02 16:00:00      629  Entry Short  ParSE  2663.90   328.7534   
2022-02-03 20:00:00      629   Exit Short  ParLE  2665.52   328.7534   
2022-02-03 20:00:00      630   Entry Long  ParLE  2665.52   333.3828   
2022-02-04 06:00:00      630    Exit Long    tpL  2798.52   333.3828   

                     Profit NONE  Profit %  Cum. Profit NONE  C

In [10]:
# combine data and trades 

df_combined  = pd.concat([df_data, df_trades]).sort_index()

df_combined = df_combined[["Close","Price","Type","Contracts"]]
print(df_combined)

# idx = df_combined.index


# for i in idx:
#     try:
#         print(len(df_combined["Type"][i]))
#     except:
#         none = 0


                       Close  Price Type  Contracts
2018-01-01 00:00:00   717.97    NaN  NaN        NaN
2018-01-01 02:00:00   734.50    NaN  NaN        NaN
2018-01-01 04:00:00   748.99    NaN  NaN        NaN
2018-01-01 06:00:00   751.99    NaN  NaN        NaN
2018-01-01 08:00:00   753.21    NaN  NaN        NaN
...                      ...    ...  ...        ...
2022-02-11 14:00:00  3102.93    NaN  NaN        NaN
2022-02-11 16:00:00  3074.83    NaN  NaN        NaN
2022-02-11 18:00:00  2992.89    NaN  NaN        NaN
2022-02-11 20:00:00  2926.34    NaN  NaN        NaN
2022-02-11 22:00:00  2927.12    NaN  NaN        NaN

[19247 rows x 4 columns]


In [12]:
# This creates the boolean for the signals
df_combined["entries"] = df_combined["Type"] == "Entry Long"
df_combined["exits"] = df_combined["Type"] == "Exit Long"
df_combined["size"] = df_combined["Contracts"]
df_combined["short_entries"] = df_combined["Type"] == "Entry Short"
df_combined["short_exits"] = df_combined["Type"] == "Exit Short"
# df_combined["Close"].to_list()

In [ ]:

#mapping out one of the sequences of the messed up trades 
#first time stamp is 2018-01-01: 14

vbt.Portfolio.from_signals([748.80, 737.02, 746.23, 759.28, 754.99], 
                            entries = [False, False, False, True, False], 
                            exits = [False, False, False, False, False], 
                            size = [1, 1, 1, 3, 1],
                            short_entries=[False, True, False, False, False],
                            short_exits=[False, False, False, True, False],
                            # price = [0, 748.27, ]
                            direction='both').positions.records_readable


# portfolio = vbt.Portfolio.from_signals(df_combined["Close"], 
#                                         entries=df_combined["entries"].to_list(), 
#                                         exits=df_combined["exits"].to_list(), 
#                                         size=df_combined["size"].to_list(),
#                                         short_entries=df_combined["short_entries"].to_list(), 
#                                         short_exits=df_combined["short_exits"].to_list(), 
#                                         price=df_combined['Price'].to_list(),
#                                         )

In [ ]:
df_combined["2018-01-01"]

# @cyrus you might want to look into the "Close" column showing NaN values when there is a trade.
My hunch it has to do with the way you concatenated the two dataframes above Note there will be three of the same timestamp because there might be an entry and an exit, plus the original closing price dataframe. 

Also, it looks like we might want to pull in the "open" price or do as you did before where you shifted the close price by one. 

I see that my prices are off because the close for 14:00:00 should really be the traded price at 16:00:00 I believe. Either that or we should also pull in the `open` prices as well as the `close` so we can better portray what is really happening.

In [13]:
# Show the Long and Short signals
print(df_combined["entries"].value_counts())
print(df_combined["short_entries"].value_counts())


False    18938
True       309
Name: entries, dtype: int64
False    18926
True       321
Name: short_entries, dtype: int64


In [15]:
# Create the portfolio object

portfolio = vbt.Portfolio.from_signals(df_combined["Close"], 
                                        entries=df_combined["entries"].to_list(), 
                                        exits=df_combined["exits"].to_list(), 
                                        size=df_combined["size"].to_list(),
                                        short_entries=df_combined["short_entries"].to_list(), 
                                        short_exits=df_combined["short_exits"].to_list(), 
                                        price=df_combined['Price'].to_list(),
                                        )

# Making vectorbt records into tradingview format so we can see if they are equal or whats going on

In [88]:
# Show the positions and trades in a readable format
records = portfolio.positions.records_readable

records = records[["Avg Entry Price", "Entry Timestamp", "Size", "Exit Timestamp", "Avg Exit Price", "Direction"]]
trade_sub = df_trades[["Type", "Price", "Contracts"]]

entries = records[["Avg Entry Price", "Entry Timestamp", "Size", "Direction"]]
exits = records[["Avg Exit Price", "Exit Timestamp", "Size", "Direction"]]

exits = exits.set_index("Exit Timestamp")
entries = entries.set_index("Entry Timestamp")

second = timedelta(seconds=1)

entries.index = entries.index + second
exits["Direction"] = "Exit " + exits["Direction"] 
entries["Direction"] = "Entry " + entries["Direction"] 

entries.columns = ["vbPrice", "vbContracts", "vbType"]
exits.columns = ["vbPrice", "vbContracts", "vbType"]

vbt_format  = pd.concat([entries, exits]).sort_index()

vbt_format = vbt_format[["vbType", "vbPrice", "vbContracts"]]

vbt_format.index = vbt_format.index.round("H")
vbt_format.index = vbt_format.index.rename("Date/Time")
# vbt_format.index = vbt_format.index - timedelta(seconds=1) if vbt_format.index.seconds == 1
# vbt_format = vbt_format.reset_index()
# vbt_format.columns = ["time", "Type", "Price", "Contracts"]


# vbt_format.loc[vbt_format.time.minutes == 1, "time"] = vbt_format.time - second
# duplicates = pd.concat([trade_sub,vbt_format]).drop_duplicates(subset = ['Price'], keep=False)
# print(duplicates)
# trade_sub["Diff?"] = trade_sub["Type"] == vbt_format["Type"]
# print(mix)
# for i in trade_sub.index:
#     print(trade_sub["Type"][i] == vbt_format["Type"][i])
print(trade_sub)
print(vbt_format)


                            Type    Price  Contracts
Date/Time                                           
2018-01-01 16:00:00  Entry Short   748.27    80.1282
2018-01-01 20:00:00   Exit Short   761.69    80.1282
2018-01-01 20:00:00   Entry Long   761.69    78.8924
2018-01-02 00:00:00    Exit Long   799.65    78.8924
2018-01-26 18:00:00   Entry Long  1061.31    57.5914
...                          ...      ...        ...
2022-01-31 16:00:00    Exit Long  2686.34   645.9219
2022-02-02 16:00:00  Entry Short  2663.90   328.7534
2022-02-03 20:00:00   Exit Short  2665.52   328.7534
2022-02-03 20:00:00   Entry Long  2665.52   333.3828
2022-02-04 06:00:00    Exit Long  2798.52   333.3828

[1260 rows x 3 columns]
                          vbType  vbPrice  vbContracts
Date/Time                                             
2018-01-01 16:00:00  Entry Short   748.27    80.128200
2018-01-01 20:00:00   Exit Short   761.69    80.128200
2018-01-01 20:00:00   Entry Long   761.69    78.892400
2018-01-02 

In [17]:
portfolio.positions.records_readable.columns

Index(['Position Id', 'Column', 'Size', 'Entry Timestamp', 'Avg Entry Price',
       'Entry Fees', 'Exit Timestamp', 'Avg Exit Price', 'Exit Fees', 'PnL',
       'Return', 'Direction', 'Status'],
      dtype='object')

In [18]:
# Show the Long and Short signals to the portfolio 
print("VectorBT thinks the portfolio has the following long and short signals: ")
print(portfolio.positions.records_readable["Direction"].value_counts()) # Check the vectorbt portfolio trades 

print("The Input Dataframe had the following information: ")
print("The Long signals are:")
print(df_combined["entries"].value_counts())

print("The Short signals are:")
print(df_combined["short_entries"].value_counts())

print("The original trades from the tradingview file are:")
print(df_trades["Type"].value_counts())

VectorBT thinks the portfolio has the following long and short signals: 
Short    318
Long     309
Name: Direction, dtype: int64
The Input Dataframe had the following information: 
The Long signals are:
False    18938
True       309
Name: entries, dtype: int64
The Short signals are:
False    18926
True       321
Name: short_entries, dtype: int64
The original trades from the tradingview file are:
Entry Short    321
Exit Short     321
Entry Long     309
Exit Long      309
Name: Type, dtype: int64


# @cyrus please look into this
It looks like we are missing 2 of our short trades. The original trade file had 321 short entries and short exits. Our VectorBT strategy thinks we only had 319 short entries. 

# Let's look at the results from VectorBT

In [ ]:
# Show the portfolio statistics and compare the returns of the banchmark
bm_rets = df_combined["Close"].vbt.to_returns() # create returns dataframe for the benchmark
portfolio.stats(settings=dict(benchmark_returns=bm_rets, # Sets the benchmark returns for stats comparisons
                              frequency='2H', # Sets the frequency for stats comparisons
                              )
                )

# @cyrus, check out the "max gross exposure" that seems like a lot of leverage
See if you can try to make sense of it all. You might need to ask for @joe's help with a spreadsheet as you try to do some forensic understanding about what is happening. 

In [ ]:
print(portfolio.stats(settings=dict(benchmark_returns=bm_rets, # Sets the benchmark returns for stats comparisons
                              frequency='2H', # Sets the frequency for stats comparisons
                              )
                )
      )
portfolio.plot().show()